# Functional Closures and Decorators

##  Closures

### Wiki says:  
In programming languages, a closure, also lexical closure or function closure, is a technique for implementing lexically scoped name binding in a language with first-class functions. Operationally, a closure is a record storing a function[a] together with an environment.[1] The environment is a mapping associating each free variable of the function (variables that are used locally, but defined in an enclosing scope) with the value or reference to which the name was bound when the closure was created.[b] Unlike a plain function, a closure allows the function to access those captured variables through the closure's copies of their values or references, even when the function is invoked outside their scope.

In [1]:
a = 'global var'
   
def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing scope')
        print(a, 'global scope')
        
    inner_func()

In [2]:
outer_func()

local to outer_func() enclosing scope
global var global scope


* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which remembers a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [15]:
def make_adder(x, z):
    # print x's address
    print('id(x): %x' % id(x))
    # enclosing scope
    print(z)
    def adder(y):
        print('in adder')
        return x + y # Python uses LEGB to find 'x'
    
    print('id(adder): %x' % id(adder))
    return adder # we return the function adder (i.e., its address in memory) 

add49 = make_adder(39, 10)
print('about to call add39')
add49(109)

id(x): 100d4a5b8
10
id(adder): 1046c3040
about to call add39
in adder


148

In [12]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
print(type(add49), repr(add49), sep='\n')

<class 'function'>
<function make_adder.<locals>.adder at 0x109720940>


In [13]:
# all functions have a closure attribute
add49.__closure__

(<cell at 0x1045a36d0: int object at 0x100d4a5b8>,)

In [8]:
# notice that the cell object has a reference to an int object
add49.__closure__[0].cell_contents

39

In [16]:
print(make_adder.__closure__)

None


* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [20]:
def logging(f):
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (f, args, kwargs))
        return f(*args, **kwargs)
    return wrapper

In [18]:
logging_add39 = logging(add49)

In [22]:
print(add49(5)) # remember that add39 just adds 39 to our argument

in adder
44


In [23]:
print(logging_add39(5))

Calling <function make_adder.<locals>.adder at 0x1046c3040>((5,), {})
in adder
44


In [24]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder(y)>

## Closure vs Classes?

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (Non-Keyword Arguments) (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [27]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

In [28]:
def add_things(a, b):
    return a + b

print('Running plain old add_things()')
print(add_things(13, 5))

Running plain old add_things()
18


In [29]:
# manual decorator assignment
cooler_add_things = document_it(add_things) 

In [30]:
print('Running cooler_add_things()')
cooler_add_things(13, 5)

Running cooler_add_things()
Running function: add_things
Positional arguments: (13, 5)
Keyword arguments: {}
Result: 18


18

In [33]:
# decorator shorthand for what we did above

# from salesforce_approved_decorator import document_it

def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

@document_it
def add_things(a, b):
    return a + b

# add_things = document_it(add_things)

print(add_things(13, -5))

Running function: add_things
Positional arguments: (13, -5)
Keyword arguments: {}
Result: 8
8


## Lab: Decorators
1. Create some function which takes an integer as its parameter
  * Create a wrapper that ensures the parameter is positive
  * use that wrapper to decorate your original function
2. Make a timer decorator that computes the elapsed time of the function wrapped by it


In [71]:
from time import time
  
  
def timer_func(func):
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__} executed in {(t2-t1)}s')
        return result
    return wrap_func

@timer_func
def long_time(n):
    for i in range(n*1000):
        pass
        

long_time(100)

Function long_time executed in 0.0010900497436523438s


## Problem with this implementation of decorators

In [39]:
	
def my_decorator(original_func):
    def wrapper(*args, **kwargs):
        """this is wrapper documentation...."""
        
        original_func(*args,**kwargs)
    return wrapper
 
def greet_hindi(name:str):
    """Greets in english"""
    print(f'Namaste, {name}!')
 
@my_decorator
def greet_french(name:str):
    """Greets in french"""
    print(f'Bonjour, {name}!')
 
greet_hindi('Philip')
greet_french('Rohan')

Namaste, Philip!
Bonjour, Rohan!


In [40]:
print(greet_hindi.__name__,greet_hindi.__doc__)
print(greet_french.__name__,greet_french.__doc__)

greet_hindi Greets in english
wrapper this is wrapper documentation....


# Using Functools

In [63]:
def logged(func):
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    with_logging.__name__ = f.__name__
    return with_logging


@logged
def f(x):
   """does some math"""
   return x + x * x

f = logged(f)
f.__name__

'f'

In [45]:

from functools import wraps
 
def a_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """A wrapper function"""
        print("called before func")
        # Extend some capabilities of func
        func()
    return wrapper
 
@a_decorator
def first_function():
    """This is docstring for first function"""
    print("first function")

In [46]:
first_function()
print(first_function.__name__,first_function.__doc__)

called before func
first function
first_function This is docstring for first function


## Passing a Parameter to decorator functions

In [62]:
def decoratory_factory(argument):
    def decorator(function):
        def wrapper(*args, **kwargs):
            print(function, argument)
            result = function(*args, **kwargs)
            return result
        return wrapper
    return decorator


In [72]:
def logged(func):
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logged
@timer_func
def dummy_func():
    print("in dummy func")

dummy_func = logged(timer_func(dummy_func))

# decorated_argumented = decoratory_factory("abc")
# print(decorated_argumented)

# @decorated_argumented
# def dummy_func_2():
#     print("in dummy func")

In [73]:
dummy_func()
# dummy_func_2()

wrap_func was called
in dummy func
Function dummy_func executed in 2.6226043701171875e-06s


In [38]:
from functools import wraps

def decorator_factory(argument):
    def a_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            """A wrapper function"""
            print("called before func with ", argument, func)
            # Extend some capabilities of func
            func()
        return wrapper
    return a_decorator


In [39]:
@decorator_factory("hello")
def sec_func():
    print("hello")
    pass

In [40]:
sec_func()

called before func with  hello <function sec_func at 0x109ea43a0>
hello


### Django using decorators https://docs.djangoproject.com/en/4.1/topics/auth/default/